## Data Cleansing and Cleaning

In [38]:
import pandas as pd 
import numpy as np
import re
import numpy as np
from sklearn.impute import KNNImputer
from collections import Counter
pd.options.mode.chained_assignment = None  # default='warn'

In [39]:
df = pd.read_csv("https://raw.githubusercontent.com/slavaspirin/Toronto-housing-price-prediction/master/houses.csv")
original_data = df.copy()
df.head()

,index,title,final_price,list_price,bedrooms,bathrooms,sqft,parking,description,mls,type,full_link,full_address,lat,long,city_district,mean_district_income,district_code
0,0,"1303 - 38 Grenville St, Toronto (C4461599) | Z...",855000,870000,2 + 1 beds,2 baths,800–899 sq. ft.,1 parking,"Luxurious And Spacious Murano Tower. 2+1, 2 Ba...",C4461599,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"38 Grenville St, Toronto , Ontario, Canada",43.661896,-79.385748,Bay Street Corridor,56526,76
1,1,"2 Cabot Crt, Toronto (W4502992) | Zoocasa",885000,898000,3 beds,2 baths,N/A sq. ft.,6 parking,Fantastic Opportunity To Live Within The Histo...,W4502992,Semi-Detached,https://www.zoocasa.com/toronto-on-real-estate...,"2 Cabot Crt, Toronto , Ontario, Canada",43.646982,-79.530356,Islington-City Centre West,52787,14
2,2,"1504 - 30 Roehampton Ave, Toronto (C4511330) |...",550000,549900,1 beds,1 baths,500–599 sq. ft.,no parking,Bright Sunfilled Spacious 1 Bdr Unit; Floor To...,C4511330,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"30 Roehampton Ave, Toronto , Ontario, Canada",43.708472,-79.397498,Mount Pleasant West,57039,104
3,3,"514 - 65 East Liberty St, Toronto (C4515763) |...",665000,600000,1 + 1 beds,1 baths,600–699 sq. ft.,1 parking,Rare Loft-Like Condo In Liberty Village W/ 18'...,C4515763,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"65 East Liberty St, Toronto , Ontario, Canada",43.638399,-79.414448,Niagara,70623,82
4,4,"61 Twelfth St, Toronto (W4519375) | Zoocasa",825513,839000,2 beds,2 baths,N/A sq. ft.,1 parking,Location! Location! Location. Your Cottage In ...,W4519375,Detached,https://www.zoocasa.com/toronto-on-real-estate...,"61 Twelfth St, Toronto , Ontario, Canada",43.597096,-79.509612,New Toronto,44101,18


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15236 entries, 0 to 15235
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 15236 non-null  int64  
 1   title                 15236 non-null  object 
 2   final_price           15236 non-null  int64  
 3   list_price            15236 non-null  int64  
 4   bedrooms              15236 non-null  object 
 5   bathrooms             15236 non-null  object 
 6   sqft                  15236 non-null  object 
 7   parking               15236 non-null  object 
 8   description           15187 non-null  object 
 9   mls                   15236 non-null  object 
 10  type                  15236 non-null  object 
 11  full_link             15236 non-null  object 
 12  full_address          15236 non-null  object 
 13  lat                   15236 non-null  float64
 14  long                  15236 non-null  float64
 15  city_district      

In [41]:
abnType = df['type'].unique()[5] ## taking out the abnormal hosue type
df = df[df['type']!= abnType]

In [42]:
## checking all the cities that are in the data

'''
we are going to split the data using the "," and then split based on teh space and get the city from each one,

This will be added to a new column called "city"
'''
df["city"] = df["full_address"].apply(lambda x: x.split(",")[1].split(" ")[1].upper())

In [43]:
df.head()

,index,title,final_price,list_price,bedrooms,bathrooms,sqft,parking,description,mls,type,full_link,full_address,lat,long,city_district,mean_district_income,district_code,city
0,0,"1303 - 38 Grenville St, Toronto (C4461599) | Z...",855000,870000,2 + 1 beds,2 baths,800–899 sq. ft.,1 parking,"Luxurious And Spacious Murano Tower. 2+1, 2 Ba...",C4461599,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"38 Grenville St, Toronto , Ontario, Canada",43.661896,-79.385748,Bay Street Corridor,56526,76,TORONTO
1,1,"2 Cabot Crt, Toronto (W4502992) | Zoocasa",885000,898000,3 beds,2 baths,N/A sq. ft.,6 parking,Fantastic Opportunity To Live Within The Histo...,W4502992,Semi-Detached,https://www.zoocasa.com/toronto-on-real-estate...,"2 Cabot Crt, Toronto , Ontario, Canada",43.646982,-79.530356,Islington-City Centre West,52787,14,TORONTO
2,2,"1504 - 30 Roehampton Ave, Toronto (C4511330) |...",550000,549900,1 beds,1 baths,500–599 sq. ft.,no parking,Bright Sunfilled Spacious 1 Bdr Unit; Floor To...,C4511330,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"30 Roehampton Ave, Toronto , Ontario, Canada",43.708472,-79.397498,Mount Pleasant West,57039,104,TORONTO
3,3,"514 - 65 East Liberty St, Toronto (C4515763) |...",665000,600000,1 + 1 beds,1 baths,600–699 sq. ft.,1 parking,Rare Loft-Like Condo In Liberty Village W/ 18'...,C4515763,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"65 East Liberty St, Toronto , Ontario, Canada",43.638399,-79.414448,Niagara,70623,82,TORONTO
4,4,"61 Twelfth St, Toronto (W4519375) | Zoocasa",825513,839000,2 beds,2 baths,N/A sq. ft.,1 parking,Location! Location! Location. Your Cottage In ...,W4519375,Detached,https://www.zoocasa.com/toronto-on-real-estate...,"61 Twelfth St, Toronto , Ontario, Canada",43.597096,-79.509612,New Toronto,44101,18,TORONTO


In [44]:
df[["bedrooms", "bathrooms","type",  "city", "parking", "sqft"]].head()

,bedrooms,bathrooms,type,city,parking,sqft
0,2 + 1 beds,2 baths,Condo Apt,TORONTO,1 parking,800–899 sq. ft.
1,3 beds,2 baths,Semi-Detached,TORONTO,6 parking,N/A sq. ft.
2,1 beds,1 baths,Condo Apt,TORONTO,no parking,500–599 sq. ft.
3,1 + 1 beds,1 baths,Condo Apt,TORONTO,1 parking,600–699 sq. ft.
4,2 beds,2 baths,Detached,TORONTO,1 parking,N/A sq. ft.


In [45]:
## let's check the uniques of the cities in the data

df["city"].value_counts()

TORONTO        15231
MISSISSAUGA        3
OUT                1
Name: city, dtype: int64

In [46]:
## now we will exclude the only row with the value "OUT" as city

df = df[df["city"] != "OUT"]
df["city"].value_counts()

TORONTO        15231
MISSISSAUGA        3
Name: city, dtype: int64

In [47]:
## now let's clean the bedrooms column

df["bedrooms"] = df["bedrooms"].apply(lambda x: ' '.join(x.split(" ")))

for i,n in enumerate(df["bedrooms"]):
    df["bedrooms"].iloc[i] = max([int(s) for s in n.split() if s.isdigit()])
df["bedrooms"].unique()

array([2, 3, 1, 0, 4, 5, 6, 8, 7, 9], dtype=object)

In [48]:
## now let's clean the bathrooms column

df["bathrooms"] = df["bathrooms"].apply(lambda x: int(''.join(x.lower()).replace("baths", '')))
df["bathrooms"].unique()

array([ 2,  1,  5,  4,  3,  6,  8,  7,  9, 14, 10, 11])

In [49]:
## now let's clean the parking column
df["parking"] = df["parking"].apply(lambda x:    str(x).replace('no parking', '0')  )
df["parking"] = df["parking"].apply(lambda x:    int(str(x).replace(' parking', '')  ))


In [50]:
df[["bedrooms", "bathrooms", "city", "parking", "list_price", "final_price"]].head()

,bedrooms,bathrooms,city,parking,list_price,final_price
0,2,2,TORONTO,1,870000,855000
1,3,2,TORONTO,6,898000,885000
2,1,1,TORONTO,0,549900,550000
3,1,1,TORONTO,1,600000,665000
4,2,2,TORONTO,1,839000,825513


In [51]:
## investigating the sqft N/A

sqft_null = df[df["sqft"] == "N/A sq. ft."]
sqft_null["sqft"].value_counts()

N/A sq. ft.    4521
Name: sqft, dtype: int64

In [52]:
sqft_null = sqft_null.query("bedrooms >= bathrooms")
sqft_null["sqft"].value_counts()

N/A sq. ft.    3916
Name: sqft, dtype: int64

In [53]:
sqft_null["bedrooms"].value_counts().sort_index()

1       8
2     565
3    2351
4     792
5     136
6      42
7       9
8       8
9       5
Name: bedrooms, dtype: int64

In [54]:
sqft_null.query("bedrooms == 1")["bathrooms"].value_counts().sort_index()

1    8
Name: bathrooms, dtype: int64

In [55]:
sqft_null.query("bedrooms == 2")["bathrooms"].value_counts().sort_index()

1    114
2    451
Name: bathrooms, dtype: int64

In [56]:
sqft_null.query("bedrooms == 3")["bathrooms"].value_counts().sort_index()

1     194
2    1480
3     677
Name: bathrooms, dtype: int64

In [57]:
sqft_null.query("bedrooms == 4")["bathrooms"].value_counts().sort_index()

1     10
2    215
3    301
4    266
Name: bathrooms, dtype: int64

In [58]:
sqft_null.query("bedrooms == 5")["bathrooms"].value_counts().sort_index()

1     1
2    20
3    42
4    46
5    27
Name: bathrooms, dtype: int64

In [59]:
sqft_null.query("bedrooms == 6")["bathrooms"].value_counts().sort_index()

1     1
2     2
3    15
4     8
5     9
6     7
Name: bathrooms, dtype: int64

In [60]:
sqft_null.query("bedrooms == 7")["bathrooms"].value_counts().sort_index()

3    1
4    4
6    3
7    1
Name: bathrooms, dtype: int64

In [61]:
sqft_null.query("bedrooms == 8")["bathrooms"].value_counts().sort_index()

3    1
4    2
6    2
7    2
8    1
Name: bathrooms, dtype: int64

In [62]:
sqft_null.query("bedrooms == 9")["bathrooms"].value_counts().sort_index()

4    1
6    2
7    1
8    1
Name: bathrooms, dtype: int64

In [63]:
## cleaning the type column

df["type"] = df["type"].str.upper()

for i,t in enumerate(df["type"]):
    if "APT" in t:
        df["type"].iloc[i] = "APT"
    elif "CONDO" in t:
        df["type"].iloc[i] = "CONDO"
    elif "DETACHED" in t:
        df["type"].iloc[i] = "DETACHED_HOUSE"
    elif "TWNHOUSE" in t:
        df["type"].iloc[i] = "TWNHOUSE"
df["type"] = df["type"][df["type"] != "COTTAGE"][df["type"] != "FOURPLEX"]

In [64]:
df = df.query("bedrooms >= bathrooms")
df

,index,title,final_price,list_price,bedrooms,bathrooms,sqft,parking,description,mls,type,full_link,full_address,lat,long,city_district,mean_district_income,district_code,city
0,0,"1303 - 38 Grenville St, Toronto (C4461599) | Z...",855000,870000,2,2,800–899 sq. ft.,1,"Luxurious And Spacious Murano Tower. 2+1, 2 Ba...",C4461599,APT,https://www.zoocasa.com/toronto-on-real-estate...,"38 Grenville St, Toronto , Ontario, Canada",43.661896,-79.385748,Bay Street Corridor,56526,76,TORONTO
1,1,"2 Cabot Crt, Toronto (W4502992) | Zoocasa",885000,898000,3,2,N/A sq. ft.,6,Fantastic Opportunity To Live Within The Histo...,W4502992,DETACHED_HOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"2 Cabot Crt, Toronto , Ontario, Canada",43.646982,-79.530356,Islington-City Centre West,52787,14,TORONTO
2,2,"1504 - 30 Roehampton Ave, Toronto (C4511330) |...",550000,549900,1,1,500–599 sq. ft.,0,Bright Sunfilled Spacious 1 Bdr Unit; Floor To...,C4511330,APT,https://www.zoocasa.com/toronto-on-real-estate...,"30 Roehampton Ave, Toronto , Ontario, Canada",43.708472,-79.397498,Mount Pleasant West,57039,104,TORONTO
3,3,"514 - 65 East Liberty St, Toronto (C4515763) |...",665000,600000,1,1,600–699 sq. ft.,1,Rare Loft-Like Condo In Liberty Village W/ 18'...,C4515763,APT,https://www.zoocasa.com/toronto-on-real-estate...,"65 East Liberty St, Toronto , Ontario, Canada",43.638399,-79.414448,Niagara,70623,82,TORONTO
4,4,"61 Twelfth St, Toronto (W4519375) | Zoocasa",825513,839000,2,2,N/A sq. ft.,1,Location! Location! Location. Your Cottage In ...,W4519375,DETACHED_HOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"61 Twelfth St, Toronto , Ontario, Canada",43.597096,-79.509612,New Toronto,44101,18,TORONTO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,17539,"1802 - 125 Peter St, Toronto (C4327974) | Zoocasa",785000,799000,2,2,700–799 sq. ft.,1,Tableau Condo - Right In The Middle Of The Ent...,C4327974,APT,https://www.zoocasa.com/toronto-on-real-estate...,"125 Peter St, Toronto , Ontario, Canada",43.648317,-79.393065,Waterfront Communities-The Island,70600,77,TORONTO
15232,17540,"179 Raleigh Ave, Toronto (E4327933) | Zoocasa",1025000,998888,5,5,3000–3500 sq. ft.,2,New Years Special. Brand New House. Beautiful ...,E4327933,DETACHED_HOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"179 Raleigh Ave, Toronto , Ontario, Canada",43.706338,-79.262789,Clairlea-Birchmount,36232,120,TORONTO
15233,17541,"68 Stagecoach Circ, Toronto (E4327881) | Zoocasa",681000,650000,3,3,N/A sq. ft.,1,Absolutely Beautiful **Freehold ** **End Unit*...,E4327881,TWNHOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"68 Stagecoach Cir, Toronto , Ontario, Canada",43.788258,-79.160342,Centennial Scarborough,54045,133,TORONTO
15234,17542,"1706 - 365 Church St, Toronto (C4330888) | Zoo...",590000,590000,1,1,500–599 sq. ft.,0,One Year New Luxury Condo Built By Menkes. Hig...,C4330888,APT,https://www.zoocasa.com/toronto-on-real-estate...,"365 Church St, Toronto , Ontario, Canada",43.660846,-79.378756,Church-Yonge Corridor,53583,75,TORONTO


## now let's create the cleaning function to be used as our base for the rest of the work

In [65]:
def cleansing_function(df):
    df = df.copy()
    abnType = df['type'].unique()[5] ## taking out the abnormal hosue type
    df = df[df['type']!= abnType]
    ## checking all the cities that are in the data

    '''
    we are going to split the data using the "," and then split based on teh space and get the city from each one,

    This will be added to a new column called "city"
    '''
    df["city"] = df["full_address"].apply(lambda x: x.split(",")[1].split(" ")[1].upper())
    ## now we will exclude the only row with the value "OUT" as city

    df = df[df["city"] != "OUT"]
    ## now let's clean the bedrooms column

    df["bedrooms"] = df["bedrooms"].apply(lambda x: ' '.join(x.split(" ")))

    for i,n in enumerate(df["bedrooms"]):
        df["bedrooms"].iloc[i] = max([int(s) for s in n.split() if s.isdigit()])
    ## now let's clean the bathrooms column

    df["bathrooms"] = df["bathrooms"].apply(lambda x: int(''.join(x.lower()).replace("baths", '')))
    ## now let's clean the parking column
    df["parking"] = df["parking"].apply(lambda x:    str(x).replace('no parking', '0')  )
    df["parking"] = df["parking"].apply(lambda x:    int(str(x).replace(' parking', '')  ))
    
    ## cleaning the type column

    df["type"] = df["type"].str.upper()

    for i,t in enumerate(df["type"]):
        if "APT" in t:
            df["type"].iloc[i] = "APT"
        elif "CONDO" in t:
            df["type"].iloc[i] = "CONDO"
        elif "DETACHED" in t:
            df["type"].iloc[i] = "DETACHED_HOUSE"
        elif "TWNHOUSE" in t:
            df["type"].iloc[i] = "TWNHOUSE"
    df["type"] = df["type"][df["type"] != "COTTAGE"][df["type"] != "FOURPLEX"]
    
    ## counting out rows that have bathrooms more than bedrooms
    df = df.query("bedrooms >= bathrooms")
    
    ## cleaning the sqft column and applying the average on each value
    sqft = df["sqft"].str.split(" ", 1).apply(lambda x: x[0])
    sqft.replace({"N/A": "0-0"}, inplace=True)
    sqft = sqft.apply(lambda x: re.split('–|-',x))

    sqft = sqft.apply(lambda x: min(int(x[0]) , int(x[1])))
    sqft.replace({0: np.nan}, inplace=True)
    df["sqft"] = sqft
    
    frames = []
    for room in list(set(df['bedrooms'])):
        df_rooms = df[df['bedrooms']== room]
        df_rooms['sqft'].fillna(df_rooms['sqft'].min(),inplace = True)
        frames.append(df_rooms)
    final_df = pd.concat(frames)
    final_df = final_df.reset_index().drop(columns=['index', 'level_0'])
    final_df["sqft"] = final_df["sqft"].round()
    df = final_df
    return df


In [66]:
df["sqft"].value_counts()

N/A sq. ft.          3916
600–699 sq. ft.      1477
500–599 sq. ft.      1123
1000–1199 sq. ft.    1071
700–799 sq. ft.       959
800–899 sq. ft.       940
1200–1399 sq. ft.     755
900–999 sq. ft.       751
1500–2000 sq. ft.     406
1100–1500 sq. ft.     374
1400–1599 sq. ft.     350
2000–2500 sq. ft.     281
0-499 sq. ft.         257
1600–1799 sq. ft.     164
2500–3000 sq. ft.     138
700–1100 sq. ft.       89
3000–3500 sq. ft.      61
3500–5000 sq. ft.      61
1800–1999 sq. ft.      58
2000–2249 sq. ft.      47
2250–2499 sq. ft.      15
2750–2999 sq. ft.       5
2500–2749 sq. ft.       3
3250–3499 sq. ft.       2
3000–3249 sq. ft.       2
900–1099 sq. ft.        2
1100–1299 sq. ft.       1
500–699 sq. ft.         1
700–899 sq. ft.         1
Name: sqft, dtype: int64

In [67]:
df.shape

(13310, 19)

In [68]:
sqft = df["sqft"].str.split(" ", 1).apply(lambda x: x[0])
sqft.replace({"N/A": "0-0"}, inplace=True)
sqft = sqft.apply(lambda x: re.split('–|-',x))

sqft = sqft.apply(lambda x: min(int(x[0]), int(x[1])))
sqft.replace({0: np.nan}, inplace=True)
df["sqft"] = sqft

In [69]:
df

,index,title,final_price,list_price,bedrooms,bathrooms,sqft,parking,description,mls,type,full_link,full_address,lat,long,city_district,mean_district_income,district_code,city
0,0,"1303 - 38 Grenville St, Toronto (C4461599) | Z...",855000,870000,2,2,800.0,1,"Luxurious And Spacious Murano Tower. 2+1, 2 Ba...",C4461599,APT,https://www.zoocasa.com/toronto-on-real-estate...,"38 Grenville St, Toronto , Ontario, Canada",43.661896,-79.385748,Bay Street Corridor,56526,76,TORONTO
1,1,"2 Cabot Crt, Toronto (W4502992) | Zoocasa",885000,898000,3,2,NaN,6,Fantastic Opportunity To Live Within The Histo...,W4502992,DETACHED_HOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"2 Cabot Crt, Toronto , Ontario, Canada",43.646982,-79.530356,Islington-City Centre West,52787,14,TORONTO
2,2,"1504 - 30 Roehampton Ave, Toronto (C4511330) |...",550000,549900,1,1,500.0,0,Bright Sunfilled Spacious 1 Bdr Unit; Floor To...,C4511330,APT,https://www.zoocasa.com/toronto-on-real-estate...,"30 Roehampton Ave, Toronto , Ontario, Canada",43.708472,-79.397498,Mount Pleasant West,57039,104,TORONTO
3,3,"514 - 65 East Liberty St, Toronto (C4515763) |...",665000,600000,1,1,600.0,1,Rare Loft-Like Condo In Liberty Village W/ 18'...,C4515763,APT,https://www.zoocasa.com/toronto-on-real-estate...,"65 East Liberty St, Toronto , Ontario, Canada",43.638399,-79.414448,Niagara,70623,82,TORONTO
4,4,"61 Twelfth St, Toronto (W4519375) | Zoocasa",825513,839000,2,2,NaN,1,Location! Location! Location. Your Cottage In ...,W4519375,DETACHED_HOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"61 Twelfth St, Toronto , Ontario, Canada",43.597096,-79.509612,New Toronto,44101,18,TORONTO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,17539,"1802 - 125 Peter St, Toronto (C4327974) | Zoocasa",785000,799000,2,2,700.0,1,Tableau Condo - Right In The Middle Of The Ent...,C4327974,APT,https://www.zoocasa.com/toronto-on-real-estate...,"125 Peter St, Toronto , Ontario, Canada",43.648317,-79.393065,Waterfront Communities-The Island,70600,77,TORONTO
15232,17540,"179 Raleigh Ave, Toronto (E4327933) | Zoocasa",1025000,998888,5,5,3000.0,2,New Years Special. Brand New House. Beautiful ...,E4327933,DETACHED_HOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"179 Raleigh Ave, Toronto , Ontario, Canada",43.706338,-79.262789,Clairlea-Birchmount,36232,120,TORONTO
15233,17541,"68 Stagecoach Circ, Toronto (E4327881) | Zoocasa",681000,650000,3,3,NaN,1,Absolutely Beautiful **Freehold ** **End Unit*...,E4327881,TWNHOUSE,https://www.zoocasa.com/toronto-on-real-estate...,"68 Stagecoach Cir, Toronto , Ontario, Canada",43.788258,-79.160342,Centennial Scarborough,54045,133,TORONTO
15234,17542,"1706 - 365 Church St, Toronto (C4330888) | Zoo...",590000,590000,1,1,500.0,0,One Year New Luxury Condo Built By Menkes. Hig...,C4330888,APT,https://www.zoocasa.com/toronto-on-real-estate...,"365 Church St, Toronto , Ontario, Canada",43.660846,-79.378756,Church-Yonge Corridor,53583,75,TORONTO


In [70]:
frames = []
for room in list(set(df['bedrooms'])):
    df_rooms = df[df['bedrooms']== room]
    df_rooms['sqft'].fillna(df_rooms['sqft'].min(),inplace = True)
    print(df_rooms['sqft'].min())
    frames.append(df_rooms)
final_df = pd.concat(frames)
final_df = final_df.reset_index().drop(columns=['index', 'level_0'])

500.0
500.0
600.0
800.0
1100.0
1500.0
2500.0
2500.0
2000.0


In [71]:
final_df["sqft"] = final_df["sqft"].round()
final_df.head()

,title,final_price,list_price,bedrooms,bathrooms,sqft,parking,description,mls,type,full_link,full_address,lat,long,city_district,mean_district_income,district_code,city
0,"1504 - 30 Roehampton Ave, Toronto (C4511330) |...",550000,549900,1,1,500.0,0,Bright Sunfilled Spacious 1 Bdr Unit; Floor To...,C4511330,APT,https://www.zoocasa.com/toronto-on-real-estate...,"30 Roehampton Ave, Toronto , Ontario, Canada",43.708472,-79.397498,Mount Pleasant West,57039,104,TORONTO
1,"514 - 65 East Liberty St, Toronto (C4515763) |...",665000,600000,1,1,600.0,1,Rare Loft-Like Condo In Liberty Village W/ 18'...,C4515763,APT,https://www.zoocasa.com/toronto-on-real-estate...,"65 East Liberty St, Toronto , Ontario, Canada",43.638399,-79.414448,Niagara,70623,82,TORONTO
2,"412 - 99 Blackwell Ave, Toronto (E4513033) | Z...",269900,269900,1,1,600.0,1,Excellent Location! Well-Kept Spacious 1 Bdrm ...,E4513033,APT,https://www.zoocasa.com/toronto-on-real-estate...,"99 Blackwell Ave, Toronto , Ontario, Canada",43.805327,-79.220229,Malvern,29573,132,TORONTO
3,"1006 - 2 Glamorgan Ave, Toronto (E4512065) | Z...",325000,299000,1,1,700.0,1,Beautiful One Bedroom Unit Is Newly Updated An...,E4512065,APT,https://www.zoocasa.com/toronto-on-real-estate...,"2 Glamorgan Ave, Toronto , Ontario, Canada",43.769495,-79.283388,Dorset Park,31692,126,TORONTO
4,"2023 - 800 Lawrence Ave W, Toronto (W4509807) ...",478000,469000,1,1,600.0,1,A Must See - Spacious 1+1 Bedroom In Near York...,W4509807,APT,https://www.zoocasa.com/toronto-on-real-estate...,"800 Lawrence Ave W, Toronto , Ontario, Canada",43.714543,-79.453072,Yorkdale-Glen Park,38527,31,TORONTO


In [72]:
# df = final_df
df.sort_values(by="sqft")["sqft"].value_counts()

600.0     1477
500.0     1124
1000.0    1071
700.0     1049
800.0      940
1200.0     755
900.0      753
1500.0     406
1100.0     375
1400.0     350
2000.0     328
1600.0     164
2500.0     141
3000.0      63
3500.0      61
1800.0      58
2250.0      15
2750.0       5
3250.0       2
Name: sqft, dtype: int64

In [73]:
df[["bedrooms", "bathrooms","type",  "city", "parking", "sqft"]].head()

,bedrooms,bathrooms,type,city,parking,sqft
0,2,2,APT,TORONTO,1,800.0
1,3,2,DETACHED_HOUSE,TORONTO,6,NaN
2,1,1,APT,TORONTO,0,500.0
3,1,1,APT,TORONTO,1,600.0
4,2,2,DETACHED_HOUSE,TORONTO,1,NaN
